# Imports
Just the standard. Jikanpy is a python wrapper for the unofficial MAL API.

In [1]:
import torch
import pandas as pd
import numpy as np
import os
from jikanpy import Jikan
jikan = Jikan()

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device, torch.cuda.get_device_name(torch.cuda.current_device())

(device(type='cuda'), 'NVIDIA GeForce RTX 3070 Laptop GPU')

# The recommender
A pipelined implementation that takes care of the boring processing work. <br>
Given a username, it outputs the top n 'seen' and 'unseen' recommendations. The 'seen' recommendations are just to see how the models thinks the user would rate everything they've already rated.<br>
It first checks if the user is already trained on, and if not, it fetches their ratings details from MAL, trains their specific weights, and then appends them to the existing user_matrix.

In [11]:
class Recommender:
    def __init__(
        self,
        anime_list,
        user_list,
        aid_mapping,
        uid_mapping,
        anime_matrix,
        user_matrix,
        a_avs,
        a_stds,
        mask
    ):
        self.al = pd.read_csv(anime_list)
        self.ul = pd.read_csv(user_list)
        self.ul_dir = user_list
        self.aidm = pd.read_csv(aid_mapping)
        self.uidm = pd.read_csv(uid_mapping)
        self.uidm_dir = uid_mapping
        self.anime_matrix = torch.load(anime_matrix)
        self.user_matrix = torch.load(user_matrix)
        self.um_dir = user_matrix
        self.a_avs = torch.load(a_avs)
        self.a_stds = torch.load(a_stds)
        self.mask = torch.load(mask).to_dense().cpu()
        self.mask_dir = mask
        self.latent_features = self.user_matrix.shape[0]
    
    def fetch_user(self, username, ntop):
        print("User not found. Fetching from MAL...")
        user = self.ul.loc[self.ul["username"]==username]
        user_id = -1
        if user.empty:
            user_id = jikan.user(username=username)['user_id']
            self.ul.loc[len(self.ul)] = {"username": username, "user_id": user_id}
            self.ul.to_csv(self.ul_dir, index=False)
        else:
            user_id = user['user_id']
            
        if type(user_id) is not int:
            user_id = list(user_id)[0]
        
        animelist = jikan.user(username=username, request='animelist')['anime']
        print("Fetched!")
        prefs = []
        for anime in animelist:
            if anime['watching_status'] == 2 and anime['score'] != 0:
                prefs.append({'anime_id': anime['mal_id'], 'my_score': anime['score']})
        prefs = pd.DataFrame(prefs)
        prefs = prefs.loc[prefs['anime_id'].isin(recom.aidm['anime_id'])]
        
        normalized = pd.merge(prefs, recom.aidm[['anime_id', 'aid']])[['aid', 'my_score']].sort_values(by='aid')
        
        new_index = pd.Index(np.arange(0,5000,1), name='aid')
        normalized = normalized.set_index('aid').reindex(new_index).reset_index().fillna(0)
        trainable = torch.tensor(normalized['my_score'].values).reshape(-1,1)
        return self.train_user_id(user_id, trainable, ntop)
    
    def train_user_id(self, user_id, scores, ntop):
        # user_id MUST be present in UserList.csv
        print("Training features for user...")
        new_user = torch.randn(self.latent_features, 1, requires_grad=True, device=device)
        new_optim = torch.optim.SGD([{'params': new_user}], lr=1e-4, momentum=0.9)
        clone_anime = self.anime_matrix
        scores = scores.to(device)
        mask = torch.where(scores>0, 1, 0)
        scores = mask * (scores - self.a_avs) / self.a_stds
        
        for epoch in range(10000):
#             if epoch%100 == 0:
#                 preds = torch.matmul(clone_anime, new_user) * self.a_stds + self.a_avs
#                 rmse = float(torch.sum(torch.square(mask * preds - mask * (scores * self.a_stds + self.a_avs))) / torch.count_nonzero(scores))
#                 rmse = np.sqrt(rmse)
#                 print(rmse)
                
            new_optim.zero_grad()    
            loss = torch.norm(mask * torch.matmul(clone_anime, new_user) - scores) + 4e-3 * torch.norm(new_user)
            loss.backward()
            new_optim.step()
        
        print("Trained! Calculating top anime...")
        self.mask = torch.cat((self.mask, mask.cpu()), dim=1)
        self.uidm.loc[len(self.uidm.index)] = [self.user_matrix.shape[1], user_id]
        self.user_matrix = torch.cat((self.user_matrix, new_user), dim=1)
        torch.save(self.user_matrix, self.um_dir)
        torch.save(self.mask.bool(), self.mask_dir)
        self.uidm.to_csv(self.uidm_dir, index=False)
        return self.get_anime_for_userfeatures(new_user, mask, ntop)
    
    def get_anime_for_userfeatures(self, user, mask, ntop):
        
        mask_seen = mask.reshape(-1,1).to(device)
        mask_unseen = 1 - mask_seen
        preds = torch.squeeze(torch.matmul(self.anime_matrix, user.reshape(-1,1))).reshape(-1,1)
        preds = preds * self.a_stds + self.a_avs
        unseen = mask_unseen * preds
        seen = mask_seen * preds
        seen = seen.to('cpu')
        unseen = unseen.to('cpu')
        
        us = pd.DataFrame(pd.DataFrame(unseen.detach().numpy()))
        us = us.reset_index()
        us.columns = ['aid', 'prediction']
        
        no_prequels = self.al.loc[~self.al['related'].astype(str).str.contains('Prequel', regex=False)][['anime_id', 'title']]
        #no_prequels = self.al[['anime_id', 'title']]
        umerged = pd.merge(pd.merge(us, self.aidm)[['prediction', 'anime_id']], no_prequels)
        udesc = umerged.sort_values(by='prediction', ascending=False)
        udesc = udesc.loc[udesc['prediction']>0].head(ntop)
        
        s = pd.DataFrame(pd.DataFrame(seen.detach().numpy()))
        s = s.reset_index()
        s.columns = ['aid', 'prediction']
        
        smerged = pd.merge(pd.merge(s, self.aidm)[['prediction', 'anime_id']], self.al[['anime_id', 'title']])
        sdesc = smerged.sort_values(by='prediction', ascending=False)
        sdesc = sdesc.loc[sdesc['prediction']>0].head(ntop)
        
        return udesc, sdesc
        
    def get_anime_for_username(self, username, ntop):
        user_id = self.ul.loc[self.ul['username'] == username]['user_id']
        if user_id.empty:
            return self.fetch_user(username, ntop)
        user_id = list(user_id)[0]
        uid = self.uidm.loc[self.uidm['user_id'] == user_id]['uid']
        if uid.empty:
            return self.fetch_user(username, ntop)
        print("User found! Calculating top anime...")
        uid = list(uid)[0]
        
        return self.get_anime_for_userfeatures(self.user_matrix[:, uid], self.mask[:, uid], ntop)
        
    

In [12]:
directory = "MAL Dataset/a5000u100k"
recom = Recommender(
    anime_list = "MAL Dataset/AnimeList.csv",
    user_list = "MAL Dataset/ul.csv",
    aid_mapping = os.path.join(directory, 'aid_mapping.csv'),
    uid_mapping = os.path.join(directory, 'uid_mapping - Copy.csv'),
    anime_matrix = os.path.join(directory, 'anime_matrix.pt'),
    user_matrix = os.path.join(directory, 'user_matrix - Copy.pt'),
    a_avs = os.path.join(directory, 'a_avs.pt'),
    a_stds = os.path.join(directory, 'a_stds.pt'),
    mask = os.path.join(directory, 'mask - Copy.pt')
)

In [13]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [14]:
u, s = recom.get_anime_for_username("Green_Fish", 20)
u

User not found. Fetching from MAL...
Fetched!
Training features for user...
Trained! Calculating top anime...


,prediction,anime_id,title
86,9.139127,5114,Fullmetal Alchemist: Brotherhood
1603,9.133775,2293,Kyoushoku Soukou Guyver
303,8.985424,918,Gintama
181,8.840632,15335,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...
472,8.725762,457,Mushishi
927,8.697750,263,Hajime no Ippo
283,8.648918,34599,Made in Abyss
6,8.644767,44,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsui...
58,8.594814,2001,Tengen Toppa Gurren Lagann
158,8.574487,12355,Ookami Kodomo no Ame to Yuki


In [15]:
s

,prediction,anime_id,title
179,9.037200,11061,Hunter x Hunter (2011)
150,8.982960,9253,Steins;Gate
340,8.871370,32281,Kimi no Na wa.
1,8.785701,19,Monster
1481,8.696426,28851,Koe no Katachi
81,8.669950,2904,Code Geass: Hangyaku no Lelouch R2
15,8.579751,199,Sen to Chihiro no Kamikakushi
60,8.555236,1575,Code Geass: Hangyaku no Lelouch
59,8.493195,1535,Death Note
14,8.486153,164,Mononoke Hime


In [18]:
u, s = recom.get_anime_for_username("dark_dragon007", 20)
u

User found! Calculating top anime...


,prediction,anime_id,title
1603,9.833142,2293,Kyoushoku Soukou Guyver
2508,9.695076,344,Futatsu no Spica
303,9.250276,918,Gintama
181,9.176454,15335,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...
472,9.030375,457,Mushishi
1,9.019193,19,Monster
2636,8.945395,24909,Kaitou Joker
0,8.916672,1,Cowboy Bebop
12,8.906732,164,Mononoke Hime
927,8.880285,263,Hajime no Ippo


In [17]:
u, s = recom.get_anime_for_username("Nightshade999", 20)
u

User not found. Fetching from MAL...
Fetched!
Training features for user...
Trained! Calculating top anime...


,prediction,anime_id,title
303,9.310429,918,Gintama
181,9.266267,15335,Gintama Movie 2: Kanketsu-hen - Yorozuya yo Ei...
266,9.226662,32281,Kimi no Na wa.
1115,9.166783,28851,Koe no Katachi
3779,9.090664,2141,Oseam
1276,9.085887,1196,Masuda Kousuke Gekijou Gag Manga Biyori
927,9.043202,263,Hajime no Ippo
472,9.029847,457,Mushishi
283,9.024250,34599,Made in Abyss
58,9.023790,2001,Tengen Toppa Gurren Lagann
